In [47]:
%matplotlib inline
import glob
import os, sys
import random
from tqdm import tqdm

import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

import pandas as pd

import matplotlib.pyplot as plt

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [62]:
train_data_dir = '/Users/student/project/train'#replace by your path
test_data_dir = '/Users/student/project/test1'#replace by your path

epochs = 20
batch_size = 20
img_width, img_height = 150, 150

training_n_bound = 5000  # set to None to use the entire training dataset

In [63]:
def gen_image_label(directory):
    ''' A generator that yields (label, id, jpg_filename) tuple.'''
    for root, dirs, files in os.walk(directory):
        for f in files:
            _, ext = os.path.splitext(f)
            if ext != '.jpg':
                continue
            basename = os.path.basename(f)
            splits = basename.split('.')
            if len(splits) == 3:
                label, id_, ext = splits
            else:
                label = None
                id_, ext = splits
            fullname = os.path.join(root, f)
            yield label, int(id_), fullname

In [64]:
lst = list(gen_image_label(train_data_dir))
random.shuffle(lst)
if training_n_bound is not None:
    lst = lst[:training_n_bound]
train_df = pd.DataFrame(lst, columns=['label', 'id', 'filename'])
train_df = train_df.sort_values(by=['label', 'id'])
train_df['label_code'] = train_df.label.map({'cat':0, 'dog':1})

train_df.head(3)

,label,id,filename,label_code
4127,cat,7,/Users/student/project/train/cat.7.jpg,0
3859,cat,14,/Users/student/project/train/cat.14.jpg,0
4026,cat,15,/Users/student/project/train/cat.15.jpg,0


In [67]:
lst = list(gen_image_label(test_data_dir))
test_df = pd.DataFrame(lst, columns=['label', 'id', 'filename'])
test_df = test_df.sort_values(by=['label', 'id'])
test_df['label_code'] = test_df.label.map({'cat':0, 'dog':1})

test_df.head(3)

,label,id,filename,label_code
0,None,1,/Users/student/project/test1/1.jpg,NaN
3612,None,2,/Users/student/project/test1/2.jpg,NaN
4723,None,3,/Users/student/project/test1/3.jpg,NaN


In [68]:
def gen_label_image_batch(df, batch_size, n_max_batch=10):
    ''' A generator that yields image as np array, batch by batch.'''
    stacked = None
    img_arrays = []
    label_arrays = []
    n_batch = 0
    for index, row in df.iterrows():
        img_arrays.append(
            img_to_array(
                load_img(row['filename'], target_size=(img_width, img_height))))
        label_arrays.append(row['label_code'])
        if len(img_arrays) % batch_size == 0:
            yield np.array(label_arrays), np.stack(img_arrays)
            n_batch += 1
            img_arrays = []
            label_arrays = []
            if n_max_batch is not None and n_batch == n_max_batch:
                break
    if img_arrays and label_arrays:
        yield np.array(label_arrays), np.stack(img_arrays)

In [69]:
datagen = ImageDataGenerator(rescale=1./255)
def gen_embedding_batch(df, batch_size, n_max_batch=None):
    ''' A generator that yields the embeddings, batch by batch 
        The embedding comes from pretrained VGG16 model.
    '''
    batches = gen_label_image_batch(df, 
                                    batch_size=batch_size, 
                                    n_max_batch=n_max_batch)
    model = applications.VGG16(include_top=False, 
                               weights='imagenet')
    for i, (label, imgs) in tqdm(enumerate(batches)):
        generator = datagen.flow(
            imgs,
            label,
            batch_size=batch_size,
            shuffle=False)
        embedding_batch = model.predict_generator(
            generator, workers=4, verbose=0)
        yield embedding_batch

In [70]:
def gen_or_load_embedding(df, saved_embedding, force_gen=False):
    if os.path.exists(saved_embedding) and not force_gen:
        print('Loading embedding from %s...' % (saved_embedding,))
        embedding = np.load(open(saved_embedding, 'rb'))
    else:
        embedding = np.stack(
            gen_embedding_batch(df, 
                                batch_size=batch_size), 
            axis=0)
        embedding = embedding.reshape(
            [embedding.shape[0] * embedding.shape[1]] + list(embedding.shape[2:]))
        np.save(open(saved_embedding, 'wb'), embedding)
    return embedding

In [71]:
train_embeddings = gen_or_load_embedding(train_df, 'train_embeddings.npy', force_gen=True)#please wait, it contains 250it
test_embeddings = gen_or_load_embedding(test_df, 'test_embeddings.npy', force_gen=False)#please wait, it contains 625it

250it [07:55,  1.90s/it]
625it [19:38,  1.89s/it]


In [72]:
[train_embeddings.shape, test_embeddings.shape]# Check embeddings' dimensions

[(5000, 4, 4, 512), (12500, 4, 4, 512)]

In [73]:
train_indices = np.nonzero((train_df.id[:train_embeddings.shape[0]] % 4 != 0).values)[0]
validate_indices = np.nonzero((train_df.id[:train_embeddings.shape[0]] % 4 == 0).values)[0]
train_labels = train_df.label_code.values[train_indices]
validation_labels = train_df.label_code.values[validate_indices]

In [74]:
embedding_fc_model = 'embedding_fc_model.h5'

In [75]:
model = Sequential()
model.add(Flatten(input_shape=train_embeddings.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [76]:
model.fit(train_embeddings[train_indices,:],
          train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(train_embeddings[validate_indices,:],
                           validation_labels))
model.save_weights(embedding_fc_model)

Train on 3796 samples, validate on 1204 samples
Epoch 1/20
3796/3796 [==============================] - 4s 990us/step - loss: 0.5494 - acc: 0.7906 - val_loss: 0.3872 - val_acc: 0.8297
Epoch 2/20
3796/3796 [==============================] - 4s 926us/step - loss: 0.3193 - acc: 0.8656 - val_loss: 0.2656 - val_acc: 0.8912
Epoch 3/20
3796/3796 [==============================] - 3s 922us/step - loss: 0.2887 - acc: 0.8865 - val_loss: 0.3768 - val_acc: 0.8588
Epoch 4/20
3796/3796 [==============================] - 4s 937us/step - loss: 0.2340 - acc: 0.9065 - val_loss: 0.2555 - val_acc: 0.8937
Epoch 5/20
3796/3796 [==============================] - 4s 932us/step - loss: 0.2179 - acc: 0.9125 - val_loss: 0.2745 - val_acc: 0.8978
Epoch 6/20
3796/3796 [==============================] - 4s 931us/step - loss: 0.1874 - acc: 0.9276 - val_loss: 0.3709 - val_acc: 0.8729
Epoch 7/20
3796/3796 [==============================] - 4s 924us/step - loss: 0.1846 - acc: 0.9244 - val_loss: 0.2638 - val_acc: 0.8995


In [77]:
from sklearn.metrics import f1_score, accuracy_score

pred_validation = model.predict(train_embeddings[validate_indices,:])

f1 = f1_score(validation_labels, pred_validation > 0.5)
acc = accuracy_score(validation_labels, pred_validation > 0.5)
(f1, acc)

(0.8897893030794165, 0.8870431893687708)

In [78]:
pred_test = model.predict(test_embeddings)
pred_test[pred_test > 0.5] = 1
pred_test[pred_test < 0.5] = 0
pred_test.shape

(12500, 1)

In [79]:
results = pd.DataFrame({'id': pd.Series(test_df.id.values[:pred_test.shape[0]]),
                        'label': pd.Series(pred_test.T[0])})
results.to_csv('submission.csv', index=False)
results.head(10)

,id,label
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,0.0
5,6,0.0
6,7,0.0
7,8,0.0
8,9,0.0
9,10,0.0
